# 04 - DBSCAN

## Introdução

O **DBSCAN** (Density-Based Spatial Clustering of Applications with Noise) é um algoritmo de clustering baseado em densidade desenvolvido por Martin Ester e colaboradores em 1996. Diferentemente do K-Means e clustering hierárquico, o DBSCAN não requer que especifiquemos o número de clusters antecipadamente e é capaz de identificar clusters de formas arbitrárias e detectar outliers (ruído).

### Características Principais do DBSCAN:

1. **Baseado em densidade**: Agrupa pontos que estão densamente empacotados
2. **Detecção de ruído**: Identifica automaticamente outliers
3. **Forma arbitrária**: Pode encontrar clusters de qualquer formato
4. **Número automático de clusters**: Não precisa especificar k antecipadamente
5. **Robusto a outliers**: Outliers não afetam a formação dos clusters

## Fundamentos Matemáticos

O DBSCAN utiliza dois parâmetros principais:
- $\varepsilon$ (eps): raio da vizinhança
- $\text{minPts}$: número mínimo de pontos para formar um cluster

### Definições Fundamentais:

**1. Vizinhança-$\varepsilon$**: Para um ponto $p$, sua vizinhança-$\varepsilon$ é definida como:
$$N_\varepsilon(p) = \{q \in D | \text{dist}(p,q) \leq \varepsilon\}$$

**2. Ponto Central (Core Point)**: Um ponto $p$ é um ponto central se:
$$|N_\varepsilon(p)| \geq \text{minPts}$$

**3. Diretamente Alcançável por Densidade**: Um ponto $q$ é diretamente alcançável por densidade a partir de $p$ se:
- $q \in N_\varepsilon(p)$ e
- $p$ é um ponto central

**4. Alcançável por Densidade**: Um ponto $q$ é alcançável por densidade a partir de $p$ se existe uma cadeia de pontos $p_1, p_2, ..., p_n$ onde $p_1 = p$ e $p_n = q$, tal que $p_{i+1}$ é diretamente alcançável por densidade a partir de $p_i$.

**5. Conectado por Densidade**: Dois pontos $p$ e $q$ são conectados por densidade se existe um ponto $o$ tal que tanto $p$ quanto $q$ são alcançáveis por densidade a partir de $o$.

### Classificação dos Pontos:

- **Core Point (Ponto Central)**: $|N_\varepsilon(p)| \geq \text{minPts}$
- **Border Point (Ponto de Fronteira)**: $|N_\varepsilon(p)| < \text{minPts}$, mas está na vizinhança de um core point
- **Noise Point (Ponto de Ruído)**: Não é core nem border point

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN as SklearnDBSCAN
from scipy.spatial.distance import pdist, squareform
from scipy.stats import mode
import pandas as pd

plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(1)

## Implementação do DBSCAN

Vamos implementar o algoritmo DBSCAN passo a passo usando apenas NumPy:

In [ ]:
class DBSCAN:
    def __init__(self, eps=0.5, min_pts=5, metric='euclidean'):
        """Inicializa o DBSCAN com os parâmetros eps e min_pts"""
        self.eps = eps
        self.min_pts = min_pts
        self.metric = metric
        self.labels_ = None
        self.core_samples_ = None
        self.n_clusters_ = 0
    
    def _calculate_distance_matrix(self, X):
        """Calcula a matriz de distâncias entre todos os pontos"""
        if self.metric == 'euclidean':
            distances = np.linalg.norm(X[:, np.newaxis] - X, axis=2)
        # elif self.metric == '...':
            # distance = ...
        else:
            raise ValueError("Métrica não suportada")
        return distances
    
    def _get_neighbors(self, point_idx, distance_matrix):
        """Encontra todos os vizinhos dentro da distância eps"""
        return np.where(distance_matrix[point_idx] <= self.eps)[0]
    
    def _expand_cluster(self, point_idx, neighbors, cluster_id, distance_matrix, visited, labels):
        """Expande o cluster a partir do ponto inicial"""
        labels[point_idx] = cluster_id
        queue = neighbors.tolist()

        while queue:
            neighbor_idx = queue.pop()

            if not visited[neighbor_idx]:
                visited[neighbor_idx] = True
                neighbor_neighbors = self._get_neighbors(neighbor_idx, distance_matrix)

                if len(neighbor_neighbors) >= self.min_pts:
                    queue.extend(neighbor_neighbors)

            if labels[neighbor_idx] == -1:
                labels[neighbor_idx] = cluster_id

    def fit(self, X):
        """Executa o algoritmo DBSCAN"""
        n_points = len(X)
        visited = np.zeros(n_points, dtype=bool)
        cluster_id = 0
        self.labels_ = np.full(n_points, -1)  # -1 = ruído
        self.core_samples_ = []

        distance_matrix = self._calculate_distance_matrix(X)

        for point_idx in range(n_points):
            if visited[point_idx]:
                continue

            visited[point_idx] = True
            neighbors = self._get_neighbors(point_idx, distance_matrix)

            if len(neighbors) >= self.min_pts:   # core point
                self.core_samples_.append(point_idx)
                self._expand_cluster(point_idx, neighbors, cluster_id, distance_matrix, visited, self.labels_)
                cluster_id += 1

        self.core_samples_ = np.array(self.core_samples_)
        self.n_clusters_ = cluster_id
        return self

    def fit_predict(self, X):
        """Executa DBSCAN e retorna os labels"""
        self.fit(X)
        return self.labels_

## Demonstração com Dados Sintéticos

Vamos criar dados sintéticos para demonstrar o funcionamento do DBSCAN:

In [ ]:
rng = np.random.default_rng(42)

# Cabeça: círculo com leve ruído
n_head = 400
theta = rng.uniform(0, 2*np.pi, n_head)
R = 10 + rng.normal(0, 0.35, n_head)
head = np.c_[R*np.cos(theta), R*np.sin(theta)]
y_head = np.full(n_head, 0)

# Olhos: dois blobs gaussianos
n_eye = 100
eye_left  = rng.normal(loc=[-3.2,  3.0], scale=[0.45, 0.45], size=(n_eye//2, 2))
eye_right = rng.normal(loc=[ 3.2,  3.0], scale=[0.45, 0.45], size=(n_eye - n_eye//2, 2))
eyes = np.vstack([eye_left, eye_right])
y_eyes = np.full(eyes.shape[0], 1)

# Boca: arco inferior com jitter (sorriso)
n_mouth = 100
phi = rng.uniform(np.deg2rad(200), np.deg2rad(340), n_mouth)  # arco de 200° a 340°
Rm = 5 + rng.normal(0, 0.22, n_mouth)
mouth = np.c_[Rm*np.cos(phi), -1 + Rm*np.sin(phi)]
mouth += rng.normal(0, [0.12, 0.15], mouth.shape)  # engrossar um pouco
y_mouth = np.full(n_mouth, 2)

# Ruído: pontos aleatórios
n_noise = 100
noise = rng.uniform(low=[-13, -13], high=[13, 13], size=(n_noise, 2))
y_noise = np.full(n_noise, -1)

# Concatenar
X_synthetic = np.vstack([head, eyes, mouth, noise])
true_labels  = np.concatenate([y_head, y_eyes, y_mouth, y_noise])

In [ ]:
# Visualizar os dados sintéticos
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
colors = ['red', 'blue', 'green', 'gray']
for i in range(4):
    if i == 3:  # ruído
        mask = true_labels == -1
        plt.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1], c=colors[i], alpha=0.6, s=30, marker='x', label='Ruído')
    else:
        mask = true_labels == i
        plt.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1], c=colors[i], alpha=0.7, s=50, label=f'Cluster {i+1}')

plt.title('Dados Sintéticos - Classes Verdadeiras')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(X_synthetic[:, 0], X_synthetic[:, 1], c='black', alpha=0.6, s=30)
plt.title('Dados Sintéticos - Sem Labels')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
dbscan = DBSCAN(eps=1.3, min_pts=7)
labels = dbscan.fit_predict(X_synthetic)

plt.figure(figsize=(6, 6))
unique_labels = np.unique(labels)
colors = plt.cm.Set1(np.linspace(0, 1, len(unique_labels)))

for k, label in enumerate(unique_labels):
    if label == -1:
        # Ruído
        mask = labels == label
        plt.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1], c='black', marker='x', s=30, alpha=0.6, label='Ruído')
    else:
        # Clusters
        mask = labels == label
        plt.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1], c=[colors[k]], s=50, alpha=0.7, label=f'Cluster {label}')

# Destacar core points
if len(dbscan.core_samples_) > 0:
    plt.scatter(X_synthetic[dbscan.core_samples_, 0], 
                X_synthetic[dbscan.core_samples_, 1],
                s=100, facecolors='none', edgecolors='black', 
                linewidth=2, alpha=0.8)

plt.tight_layout()
plt.show()

## Aplicando DBSCAN aos Dados Sintéticos

Agora vamos aplicar nosso algoritmo DBSCAN:

In [ ]:
# Testar diferentes valores de eps e min_pts
eps_values = [0.5, 1.3, 2.0]
min_pts_values = [3, 5, 8]

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
fig.suptitle('DBSCAN com Diferentes Parâmetros', fontsize=16)

for i, eps in enumerate(eps_values):
    for j, min_pts in enumerate(min_pts_values):
        # Aplicar DBSCAN
        dbscan = DBSCAN(eps=eps, min_pts=min_pts)
        labels = dbscan.fit_predict(X_synthetic)
        
        # Visualizar resultados
        ax = axes[i, j]
        
        unique_labels = np.unique(labels)
        colors = plt.cm.Set1(np.linspace(0, 1, len(unique_labels)))
        
        for k, label in enumerate(unique_labels):
            if label == -1:
                # Ruído
                mask = labels == label
                ax.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1], 
                          c='black', marker='x', s=30, alpha=0.6, label='Ruído')
            else:
                # Clusters
                mask = labels == label
                ax.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1], 
                          c=[colors[k]], s=50, alpha=0.7, label=f'Cluster {label}')
        
        # Destacar core points
        if len(dbscan.core_samples_) > 0:
            ax.scatter(X_synthetic[dbscan.core_samples_, 0], 
                      X_synthetic[dbscan.core_samples_, 1],
                      s=100, facecolors='none', edgecolors='black', 
                      linewidth=2, alpha=0.8)
        
        ax.set_title(f'eps={eps}, min_pts={min_pts}\n{dbscan.n_clusters_} clusters')
        ax.set_xlabel('Feature 1')
        ax.set_ylabel('Feature 2')
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Análise com Parâmetros Ótimos

Vamos escolher os parâmetros que melhor capturam a estrutura dos dados:

In [ ]:
# Parâmetros que parecem funcionar melhor
best_eps = 1.3
best_min_pts = 5

# Aplicar DBSCAN com os melhores parâmetros
dbscan_best = DBSCAN(eps=best_eps, min_pts=best_min_pts)
labels_best = dbscan_best.fit_predict(X_synthetic)

# Análise detalhada
unique_labels = np.unique(labels_best)
n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)
n_noise = np.sum(labels_best == -1)
n_core_samples = len(dbscan_best.core_samples_)

print(f"Resultados do DBSCAN (eps={best_eps}, min_pts={best_min_pts}):")
print(f"- Número de clusters encontrados: {n_clusters}")
print(f"- Número de pontos de ruído: {n_noise}")
print(f"- Número de core samples: {n_core_samples}")
print(f"- Labels únicos: {unique_labels}")

In [ ]:
# Classificar pontos por tipo
core_mask = np.zeros(len(X_synthetic), dtype=bool)
if len(dbscan_best.core_samples_) > 0:
    core_mask[dbscan_best.core_samples_] = True

border_mask = (labels_best != -1) & (~core_mask)
noise_mask = labels_best == -1

print(f"Classificação dos pontos:")
print(f"- Core points: {np.sum(core_mask)}")
print(f"- Border points: {np.sum(border_mask)}")
print(f"- Noise points: {np.sum(noise_mask)}")

In [ ]:
# Visualização detalhada dos tipos de pontos
plt.figure(figsize=(15, 5))

# Subplot 1: Clusters encontrados
plt.subplot(1, 3, 1)
colors = plt.cm.Set1(np.linspace(0, 1, max(len(unique_labels), 3)))

for i, label in enumerate(unique_labels):
    if label == -1:
        mask = labels_best == label
        plt.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1], 
                   c='black', marker='x', s=50, alpha=0.8, label='Ruído')
    else:
        mask = labels_best == label
        plt.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1], 
                   c=[colors[i]], s=60, alpha=0.8, label=f'Cluster {label}')

plt.title(f'DBSCAN - Clusters Encontrados\n{n_clusters} clusters, {n_noise} ruído')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: Tipos de pontos
plt.subplot(1, 3, 2)
plt.scatter(X_synthetic[core_mask, 0], X_synthetic[core_mask, 1], 
           c='red', s=80, alpha=0.8, label='Core Points', marker='o')
plt.scatter(X_synthetic[border_mask, 0], X_synthetic[border_mask, 1], 
           c='blue', s=60, alpha=0.8, label='Border Points', marker='s')
plt.scatter(X_synthetic[noise_mask, 0], X_synthetic[noise_mask, 1], 
           c='black', s=40, alpha=0.8, label='Noise Points', marker='x')

plt.title('Classificação dos Pontos')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 3: Comparação com ground truth
plt.subplot(1, 3, 3)
for i in range(4):
    if i == 3:  # ruído
        mask = true_labels == -1
        plt.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1], 
                   c='gray', alpha=0.6, s=30, marker='x', label='Ruído Real')
    else:
        mask = true_labels == i
        plt.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1], 
                   c=colors[i], alpha=0.7, s=50, label=f'Cluster Real {i+1}')

plt.title('Ground Truth')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Estimativa do Parâmetro $\varepsilon$ usando K-Distance

Uma das maiores dificuldades do DBSCAN é escolher o valor apropriado para o parâmetro $\varepsilon$ (eps). O método **K-Distance** é uma heurística eficaz para estimar este parâmetro.

O método K-Distance consiste em:

1. **Calcular a k-ésima distância mais próxima** para cada ponto no dataset
2. **Ordenar essas distâncias** em ordem decrescente
3. **Identificar o "cotovelo"** no gráfico resultante

A intuição é que pontos dentro de clusters densos terão k-ésimas distâncias pequenas, enquanto pontos de ruído ou em bordas de clusters terão distâncias maiores.

### Algoritmo K-Distance:

Para um dataset $D$ e parâmetro $k = \text{minPts} - 1$:

1. Para cada ponto $p_i \in D$:
   - Calcule $d_k(p_i)$ = distância ao k-ésimo vizinho mais próximo
2. Ordene os valores $d_k(p_i)$
3. Plote o gráfico K-Distance
4. Escolha $\varepsilon$ no ponto onde a curva tem maior curvatura (cotovelo)

In [ ]:
from sklearn.neighbors import NearestNeighbors

def plot_k_distance(X, min_pts, title="K-Distance Plot"):
    """Plota o gráfico K-Distance usando sklearn.NearestNeighbors."""
    k = int(min_pts - 1)

    nn = NearestNeighbors(n_neighbors=k+1, metric="euclidean")
    nn.fit(X)
    distances, _ = nn.kneighbors(X)

    kth_distances = distances[:, k]
    k_distances_sorted = np.sort(kth_distances)

    # Plot
    plt.figure(figsize=(10, 6))
    plt.plot(range(len(k_distances_sorted)), k_distances_sorted, linewidth=2, label=f'{k}-distance')
    plt.xlabel("Pontos ordenados por distância")
    plt.ylabel(f"{k}-distance")
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
plot_k_distance(X_synthetic, min_pts=5, title="K-Distance Plot para Dados Sintéticos")

## Vantagens e Desvantagens do DBSCAN

### Vantagens:

1. **Não requer especificar o número de clusters antecipadamente**
2. **Pode encontrar clusters de forma arbitrária** (não apenas esféricos)
3. **Identifica automaticamente outliers/ruído**
4. **Robusto a outliers** (não afetam a formação dos clusters)
5. **Determinístico** (sempre produz os mesmos resultados)

### Desvantagens:

1. **Sensível aos parâmetros** eps e min_pts
2. **Dificuldade com clusters de densidades diferentes**
3. **Problemas em alta dimensionalidade** ("curse of dimensionality")
4. **Complexidade computacional** O(n²) no pior caso
5. **Requer escolha cuidadosa da métrica de distância**

## Exercícios

### Exercício 1: Ajuste de Parâmetros no DBSCAN em 3D

Com os dados das **três esferas concêntricas**, realize:

1. Plotar o K-Distance para diferentes valores de `min_pts` e sugerir um intervalo adequado para `eps`.
2. Selecionar os melhores parâmetros de `min_pts` e `eps`.
3. Visualizar em 3D os clusters encontrados (cores diferentes) e comentar a escolha de `eps` e `min_samples`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN


def k_distance(X, k):
    nbrs = NearestNeighbors(n_neighbors=k).fit(X)
    dists, _ = nbrs.kneighbors(X)
    return np.sort(dists[:, -1])

def plot_kdist_for_minpts(X, minpts_list):
    plt.figure(figsize=(8,5))
    for k in minpts_list:
        kd = k_distance(X, k)
        plt.plot(kd, label=f'min_pts={k}', linewidth=1.6)
    plt.title("K-Distance para diferentes min_pts")
    plt.xlabel("Pontos ordenados")
    plt.ylabel("Distância")
    plt.legend()
    plt.grid(True, alpha=.3)
    plt.show()

# Faixa típica em 3D: min_pts entre 6 e 12
plot_kdist_for_minpts(X_spheres, [6, 8, 10, 12])

# Sugestão inicial de eps pelo cotovelo
def knee_interval(kdist, low_q=0.85, high_q=0.95):
    return np.quantile(kdist, low_q), np.quantile(kdist, high_q)

kdist_ref = k_distance(X_spheres, 10)
eps_lo, eps_hi = knee_interval(kdist_ref)
print(f"Intervalo sugerido para eps ≈ [{eps_lo:.3f}, {eps_hi:.3f}]")

def avalia_dbscan(X, eps, min_samples):
    model = DBSCAN(eps=eps, min_samples=min_samples)
    labels = model.fit_predict(X)
    n_noise = np.sum(labels == -1)
    k = len(set(labels) - {-1})
    return labels, k, n_noise / len(X)

eps_grid = np.linspace(eps_lo, eps_hi, 11)
minpts_grid = [6, 8, 10, 12]

resultados = []
for m in minpts_grid:
    for e in eps_grid:
        labels, k, noise = avalia_dbscan(X_spheres, e, m)
        resultados.append((m, e, k, noise))

resultados_ordenados = sorted(resultados, key=lambda t: (abs(t[2]-3), t[3], t[1]))
best_min, best_eps, best_k, best_noise = resultados_ordenados[0]
labels_best, _, _ = avalia_dbscan(X_spheres, best_eps, best_min)

print(f"Melhor escolha: min_samples={best_min}, eps={best_eps:.3f} → clusters={best_k}, ruído={100*best_noise:.1f}%")



In [ ]:
import plotly.express as px
import pandas as pd

df_plot = pd.DataFrame(X_spheres, columns=["x","y","z"])
df_plot["cluster"] = labels_best.astype(str)

fig = px.scatter_3d(
    df_plot, x="x", y="y", z="z",
    color="cluster",
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig.update_traces(marker=dict(size=3))
fig.show()

### Exercício 2: DBSCAN com distância radial

Usando os dados das **3 esferas concêntricas** do exercício anterior:

1. Implemente a **distância radial** e use-a no DBSCAN. A **distância radial** entre dois pontos \(x_i\) e \(x_j\) é a diferença absoluta entre suas distâncias à origem: $d_{\text{radial}}(x_i, x_j) = \big|\;\|x_i\|_2 - \|x_j\|_2\;\big|$
2. Plote o **K-Distance radial** para sugerir `eps`.  
3. Teste combinações de `eps` e `min_samples`.  
4. Visualize em 3D os clusters obtidos e compare com o resultado usando distância euclidiana.  
5. Comente brevemente qual configuração foi melhor e por quê a métrica radial ajuda nesse dataset.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import plotly.express as px

assert 'X_spheres' in globals(), "Execute antes o bloco do Exercício 1 que define X_spheres (padronizado)."

# ---------- Definições ----------
def radial_metric(u, v):
    # d_radial(u,v) = | ||u||_2 - ||v||_2 |
    return abs(np.linalg.norm(u) - np.linalg.norm(v))

def k_distance_custom(X, k, metric, algorithm='brute'):
    nn = NearestNeighbors(n_neighbors=k, metric=metric, algorithm=algorithm)
    nn.fit(X)
    dists, _ = nn.kneighbors(X)
    return np.sort(dists[:, -1])

def knee_interval(kdist, low_q=0.85, high_q=0.95):
    return np.quantile(kdist, low_q), np.quantile(kdist, high_q)

def avalia_dbscan(X, eps, min_samples, metric):
    mdl = DBSCAN(eps=eps, min_samples=min_samples, metric=metric)
    labels = mdl.fit_predict(X)
    n_noise = int(np.sum(labels == -1))
    k = len(set(labels) - {-1})
    noise_rate = n_noise / len(X)
    return labels, k, noise_rate

# ---------- 1) K-Distance (métrica radial) e sugestão de eps ----------
minpts_list = [6, 8, 10, 12]  # faixa estável em 3D
k_ref = 10

kdist_curvas = []
for k in minpts_list:
    kd = k_distance_custom(X_spheres, k, metric=radial_metric)  # radial
    kdist_curvas.append(pd.DataFrame({"ordem": np.arange(len(kd)), "k_distance": kd, "min_pts": str(k)}))

df_kd = pd.concat(kdist_curvas, ignore_index=True)

# Plot K-Distance radial (todas as curvas)
fig_kd = px.line(
    df_kd, x="ordem", y="k_distance", color="min_pts",
    title="K-Distance (métrica radial) para diferentes min_pts"
)
fig_kd.show()

# Intervalo sugerido de eps pela curva com k_ref
kdist_ref_radial = k_distance_custom(X_spheres, k_ref, metric=radial_metric)
eps_lo_rad, eps_hi_rad = knee_interval(kdist_ref_radial, 0.85, 0.95)
print(f"[RADIAL] Intervalo sugerido para eps (min_pts={k_ref}): [{eps_lo_rad:.3f}, {eps_hi_rad:.3f}]")

# ---------- 2) Grade de (eps, min_samples) — métrica radial ----------
eps_grid_radial = np.linspace(eps_lo_rad, eps_hi_rad, 11)
res_radial = []
for m in minpts_list:
    for e in eps_grid_radial:
        labels, k, noise = avalia_dbscan(X_spheres, e, m, metric=radial_metric)
        res_radial.append((m, float(e), int(k), float(noise)))
res_radial_sorted = sorted(res_radial, key=lambda t: (abs(t[2]-3), t[3], t[1]))

best_min_rad, best_eps_rad, best_k_rad, best_noise_rad = res_radial_sorted[0]
labels_radial, _, _ = avalia_dbscan(X_spheres, best_eps_rad, best_min_rad, metric=radial_metric)

print("Top 5 (RADIAL):")
for r in res_radial_sorted[:5]:
    print(f"min_samples={r[0]:>2} | eps={r[1]:.3f} | clusters={r[2]} | ruído={100*r[3]:.1f}%")
print(f">>> Escolha RADIAL: min_samples={best_min_rad}, eps={best_eps_rad:.3f} "
      f"→ clusters={best_k_rad}, ruído={100*best_noise_rad:.1f}%")

# ---------- 3) Baseline EUCLIDIANA (mesma lógica de busca) ----------
# K-Distance euclidiana p/ eps
kdist_ref_eucl = k_distance_custom(X_spheres, k_ref, metric='euclidean', algorithm='auto')
eps_lo_eucl, eps_hi_eucl = knee_interval(kdist_ref_eucl, 0.85, 0.95)
print(f"[EUCLIDIANA] Intervalo sugerido para eps (min_pts={k_ref}): [{eps_lo_eucl:.3f}, {eps_hi_eucl:.3f}]")

eps_grid_eucl = np.linspace(eps_lo_eucl, eps_hi_eucl, 11)
res_eucl = []
for m in minpts_list:
    for e in eps_grid_eucl:
        labels, k, noise = avalia_dbscan(X_spheres, e, m, metric='euclidean')
        res_eucl.append((m, float(e), int(k), float(noise)))
res_eucl_sorted = sorted(res_eucl, key=lambda t: (abs(t[2]-3), t[3], t[1]))

best_min_eucl, best_eps_eucl, best_k_eucl, best_noise_eucl = res_eucl_sorted[0]
labels_euclid, _, _ = avalia_dbscan(X_spheres, best_eps_eucl, best_min_eucl, metric='euclidean')

print("Top 5 (EUCLIDIANA):")
for r in res_eucl_sorted[:5]:
    print(f"min_samples={r[0]:>2} | eps={r[1]:.3f} | clusters={r[2]} | ruído={100*r[3]:.1f}%")
print(f">>> Escolha EUCLIDIANA: min_samples={best_min_eucl}, eps={best_eps_eucl:.3f} "
      f"→ clusters={best_k_eucl}, ruído={100*best_noise_eucl:.1f}%")

# ---------- 4) Visualizações 3D (Plotly) ----------
# RADIAL
df_plot_rad = pd.DataFrame(X_spheres, columns=["x","y","z"])
df_plot_rad["cluster"] = labels_radial.astype(str)
fig_rad = px.scatter_3d(
    df_plot_rad, x="x", y="y", z="z",
    color="cluster",
    title=f"DBSCAN (métrica radial) | eps={best_eps_rad:.3f}, min_samples={best_min_rad}",
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_rad.update_traces(marker=dict(size=3))
fig_rad.show()

# EUCLIDIANA
df_plot_eu = pd.DataFrame(X_spheres, columns=["x","y","z"])
df_plot_eu["cluster"] = labels_euclid.astype(str)
fig_eu = px.scatter_3d(
    df_plot_eu, x="x", y="y", z="z",
    color="cluster",
    title=f"DBSCAN (euclidiana) | eps={best_eps_eucl:.3f}, min_samples={best_min_eucl}",
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_eu.update_traces(marker=dict(size=3))
fig_eu.show()

# ---------- Comentário ----------
print("\n 5) Comentário:")
print("Foi usada a distância radial no DBSCAN, que compara só o raio de cada ponto em relação à origem. Isso ajudou a separar bem as três esferas concêntricas, formando 3 clusters corretos e com pouco ruído.")
print("Quando testada a distância euclidiana, às vezes pontos de esferas diferentes acabavam juntos (se estavam próximos pelo ângulo), e isso confundia os agrupamentos. Já a métrica radial “ignora o ângulo” e olha apenas para o raio, que é justamente a característica que distingue as cascas.")
print("A melhor configuração ficou com min_samples entre 8 e 12 e eps dentro do intervalo indicado pelo gráfico de K-Distance (entre os percentis 85 e 95). Assim consegui separar os três grupos como esperado.")


### Exercício 3: Detecção de Anomalias com DBSCAN e DTW

O **DTW (Dynamic Time Warping)** mede a similaridade entre séries temporais mesmo quando estão defasadas ou com velocidades diferentes, alinhando-as de forma elástica. Isso permite detectar padrões semelhantes sem que a defasagem atrapalhe.

Pode ser calculado por:
```python
from dtaidistance import dtw

n = len(X)              # número de séries
D = np.zeros((n, n))    # matriz de distâncias

for i in range(n):
    for j in range(i+1, n):
        dist = dtw.distance_fast(X[i], X[j])  # distância DTW
        D[i, j] = D[j, i] = dist              # matriz simétrica
````

**Tarefas:**
1. Use o dataset de senóides com variação e **anomalias simuladas**.  
2. Adicione a métrica DTW no DBSCAN.
3. Experimente diferentes valores de `eps` e `min_samples` até que o modelo consiga separar bem séries normais das anômalas.  
4. Plote todas as séries, usando uma cor para as normais e outra para as anomalias detectadas (`label = -1`).  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

# 1) Dataset de senóides com anomalias
def generate_time_series_dataset(n_series=50, length=100, noise=0.1, n_outliers=2, random_state=42):
    rng = np.random.default_rng(random_state)
    X, y = [], []
    t = np.linspace(0, 4*np.pi, length)

    # séries normais: senóide com amplitude e frequência ligeiramente diferentes
    for _ in range(n_series):
        amp = rng.uniform(0.8, 1.2)         # amplitude
        freq = rng.uniform(0.9, 1.1)        # frequência
        phase = rng.uniform(0, 0.5*np.pi)   # pequena defasagem
        series = amp * np.sin(freq * t + phase) + noise * rng.normal(size=length)
        X.append(series); y.append(0)  # normal

    # outliers: picos ou deslocamentos fortes
    for _ in range(n_outliers):
        amp = rng.uniform(1.5, 2.0)         # amplitude anômala
        freq = rng.uniform(1.2, 1.5)        # frequência anômala
        series = amp * np.sin(freq * t) + noise * rng.normal(size=length)
        if rng.random() < 0.5:
            series[length//2] += 3  # pico
        else:
            series += rng.normal(2.0, 0.5)  # deslocamento
        X.append(series); y.append(-1)  # anomalia

    return np.array(X), np.array(y), t

X, y_true, t = generate_time_series_dataset()

# 2) Distância DTW
USING_LIB = False
try:
    from dtaidistance import dtw as _dtw
    def dtw_distance(a, b):
        return float(_dtw.distance_fast(a, b))
    USING_LIB = True
except Exception:
    def dtw_distance_window(a, b, w):
        n = len(a); m = len(b)
        w = max(w, abs(n - m))
        INF = 1e18
        D = np.full((n+1, m+1), INF, dtype=float)
        D[0, 0] = 0.0
        for i in range(1, n+1):
            ai = a[i-1]
            j0 = max(1, i - w)
            j1 = min(m, i + w)
            for j in range(j0, j1+1):
                cost = abs(ai - b[j-1])
                D[i, j] = cost + min(D[i-1, j], D[i, j-1], D[i-1, j-1])
        return float(D[n, m])
    def dtw_distance(a, b):
        # janela de ~10% do comprimento para acelerar
        return dtw_distance_window(a, b, w=max(6, len(a)//10))

print("DTW usado:", "dtaidistance.distance_fast" if USING_LIB else "DTW com janela (NumPy)")

# 3) Matriz de distâncias DTW
n = len(X)
D = np.zeros((n, n), dtype=float)
for i in range(n):
    for j in range(i+1, n):
        d = dtw_distance(X[i], X[j])
        D[i, j] = D[j, i] = d

# 4) Busca simples de (eps, min_samples) para separar normais vs anômalas
du = D[np.triu_indices(n, 1)]
q70, q95 = np.quantile(du, 0.70), np.quantile(du, 0.95)
eps_grid = np.linspace(q70, q95, 6)
min_grid = [3, 4, 5]

def f1_anom(labels, truth):
    tp = np.sum((labels == -1) & (truth == -1))
    fp = np.sum((labels == -1) & (truth ==  0))
    fn = np.sum((labels != -1) & (truth == -1))
    if tp == 0: return 0.0
    prec = tp / (tp + fp) if (tp + fp) else 0.0
    rec  = tp / (tp + fn) if (tp + fn) else 0.0
    return 0.0 if (prec + rec) == 0 else 2 * prec * rec / (prec + rec)

best = None
for ms in min_grid:
    for eps in eps_grid:
        lbls = DBSCAN(eps=eps, min_samples=ms, metric='precomputed').fit_predict(D)
        f1 = f1_anom(lbls, y_true)
        n_anom = int(np.sum(lbls == -1))
        n_clusters = len(set(lbls) - {-1})
        score = (f1, -abs(n_anom - int(np.sum(y_true == -1))), -n_clusters)
        if (best is None) or (score > best[0]):
            best = (score, ms, float(eps), lbls, f1, n_anom, n_clusters)

_, best_ms, best_eps, labels, f1_best, n_anom, n_clusters = best

print(f"Escolha final -> eps={best_eps:.3f}, min_samples={best_ms} | clusters={n_clusters} | anomalias={n_anom} | F1_anom={f1_best:.3f}")

# 5) Plot: todas as séries (azul = normais, vermelho = anomalias detectadas)
plt.figure(figsize=(12, 5))
first_norm = True; first_anom = True
for i, serie in enumerate(X):
    if labels[i] == -1:
        plt.plot(t, serie, c='red', alpha=0.9, lw=1.2, label='anomalia' if first_anom else None)
        first_anom = False
    else:
        plt.plot(t, serie, c='blue', alpha=0.35, lw=0.9, label='normal' if first_norm else None)
        first_norm = False
plt.title(f"DBSCAN + DTW | eps={best_eps:.3f}, min_samples={best_ms}")
plt.legend()
plt.tight_layout()
plt.show()
